## Day 11: Chronal Charge

https://adventofcode.com/2018/day/11

### Part 1

Following the recipe on the packet.

In [1]:
def hundreds_digit(x):
    return (x - (x // 1000) * 1000) // 100


def power_level(x, y, serial):
    rack_id = x + 10
    power_level = (rack_id * y + serial) * rack_id
    return(hundreds_digit(power_level)) - 5


assert power_level(3, 5, 8) == 4
assert power_level(122, 79, 57) == -5
assert power_level(217, 196, 39) == 0
assert power_level(101, 153, 71) == 4

In [2]:
from itertools import product


def grid(serial_number):
    g = {}

    for x in range(1, 301):
        for y in range(1, 301):
            g[(x, y)] = power_level(x, y, serial_number)

    return g


def n_by_n_coords(x, y, n=3):
    return product(range(x, x + n), range(y, y + n))
            

def most_powerful_square(grid):
    squares = product(range(1, 299), range(1, 299))
    
    return max(squares,
               key=lambda sq: sum(grid[p] 
                                  for p in n_by_n_coords(sq[0], sq[1])))

In [3]:
assert most_powerful_square(grid(18)) == (33, 45)
assert most_powerful_square(grid(42)) == (21, 61)

In [4]:
%time most_powerful_square(grid(9810))

CPU times: user 289 ms, sys: 0 ns, total: 289 ms
Wall time: 288 ms


(245, 14)

### Part 2

That took some thinking about, though using itertools rather than nested `for`s halved the time. Thinking about the next part, using `numpy` would have been easier in the first place. Brute force again.

In [5]:
import numpy as np


# Define a 301x301 grid for convenience
def grid_np(serial_number):
    return np.array([[power_level(x, y, serial_number) 
                      for y in range(301)] 
                     for x in range(301)])


# Need to transpose to make the visualisation equivalent
# to the one in the problem statement
grid_np(18)[32:37, 44:48].T

array([[-2, -4,  4,  4,  4],
       [-4,  4,  4,  4, -5],
       [ 4,  3,  3,  4, -4],
       [ 1,  1,  2,  4, -3]])

In [6]:
def most_powerful_square_np(grid):
    return max((grid[x:x+3, y:y+3].sum(), x, y) 
                for x, y in product(range(1, 299), range(1, 299)))

most_powerful_square_np(grid_np(18))

(29, 33, 45)

In [7]:
%time most_powerful_square(grid(9810))

CPU times: user 285 ms, sys: 0 ns, total: 285 ms
Wall time: 286 ms


(245, 14)

Surprisingly that makes no difference.

In [8]:
def most_powerful_square_of_all_np(grid):
    maxes = []
    return max(max((grid[x:(x+n), y:(y+n)].sum(), x, y, n) 
                    for x, y in product(range(1, 302 - n), range(1, 302 - n)))
               for n in range(1, 301))[1:]

Debugging this took so long that I switched to pypy.

```
$ time pypy3 chronal_change.py 
(235, 206, 13)

real	5m59.088s
user	5m57.292s
sys	0m1.181s
```

### Post-mortem

Apparently I should have been using a [summed-area table](https://en.wikipedia.org/wiki/Summed-area_table). That's pretty smart, let's give it a go.

In [9]:
from collections import defaultdict

def mpsoa_quick(grid):
    sat = defaultdict(int)
    for x in range(1, 301):
        for y in range(1, 301):
            sat[(x, y)] = grid[(x, y)] + sat[(x - 1, y)] \
                + sat[(x, y - 1)] - sat[(x - 1, y - 1)]
        
    def area_power(x, y, n):
        return sat[(x + n - 1, y + n - 1)] \
                + sat[(x - 1), (y - 1)] \
                - sat[(x + n - 1, y - 1)] \
                - sat[(x - 1, y + n - 1)]
                
    return max(max((area_power(x, y, n), x, y, n) 
                   for x, y in product(range(1, 302 - n), 
                                       range(1, 302 - n)))
               for n in range(1, 301))[1:]

In [10]:
%time mpsoa_quick(grid(18))

CPU times: user 9.73 s, sys: 10.8 ms, total: 9.74 s
Wall time: 9.79 s


(90, 269, 16)

In [11]:
%time mpsoa_quick(grid(9810))

CPU times: user 9.69 s, sys: 24 ms, total: 9.71 s
Wall time: 9.71 s


(235, 206, 13)

That's better.